In [ ]:
from __future__ import print_function
import os
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, fixed
from IPython.display import Image
import matplotlib.pyplot as plt
from labellines import labelLine, labelLines
from cycler import cycler

import sys
import time
import numpy as np


In [ ]:
def ThermalEff(Comp_eff, Turb_eff, T4qT2, PR, gamma = 1.4):
    k = (gamma - 1)/gamma
    A = PR**k
    Eff_th = (Turb_eff*T4qT2*(1-1/A) - (A-1)/Comp_eff)/(T4qT2 - 1 - (A-1)/Comp_eff)
    
    return Eff_th
    
def Work_per_flow(Comp_eff, Turb_eff, T4qT2, PR, gamma = 1.4, turb_comp_works = False):
    """
    Calculates the Wnet/(mair*cp*Tt2)
    """
    k = (gamma - 1)/gamma
    A = PR**k
    
    AvailableWork = Turb_eff*T4qT2*(1-1/A)
    CompWork = (A-1)/Comp_eff
    Tt3qTt2 = (A-1)/Comp_eff + 1
    
    Wqflow = AvailableWork - CompWork
    
    if turb_comp_works:
        return Wqflow, AvailableWork, CompWork, Tt3qTt2
    else:
        return Wqflow
    
def Work_per_flow_w_PT(Comp_eff, Turb_eff, PowerTurb_eff, T4qT2, PR, gamma = 1.4, details = False):
    """
    Calculates the Wnet/(mair*cp*Tt2) with a power turbine
    """
    
    k = (gamma - 1)/gamma
    A = PR**k
    
    CompWqflow = (A-1)/Comp_eff # W_compr/(m_air*cp*Tt2) = (PR**(k) - 1)/eta  
    T5qT4      = 1 + CompWqflow/T4qT2
    T5qT4_is   = 1 + (T5qT4 - 1)/Turb_eff 
    P5qP4      = T5qT4_is**(1/k)
    P4qP5      = 1/P5qP4
    P6qP5      = P4qP5/PR
    
    T6qT5      = 1 + PowerTurb_eff*(P6qP5**k-1)
    
    Wqflow = T4qT2*(1 - T6qT5*T5qT4) - CompWqflow
#     Wqflow = T4qT2*(1-
#                     (1+PowerTurb_eff*
#                      (1-1/A*(1/(1-(A-1)/(Turb_eff*Comp_eff*T4qT2))))*(1-(A-1)/(T4qT2*Comp_eff))
#                     )
#                     -CompWqflow)

    
    if details:
        return Wqflow, T5qT4
    else:
        return Wqflow


In [ ]:
PR = np.linspace(1,60,100)

fig, ax = plt.subplots(1,4,figsize=(20,5),dpi =200, sharex = False)

cc = cycler(linestyle=['-', '--', '-.'])

eta_c = 0.9
eta_t = 0.9
# for a in ax:
#     a.set_prop_cycle(cc)

for TR in np.linspace(4,7,3):

    ax[0].plot(PR, ThermalEff(eta_c,eta_t,TR,PR), label = round(TR,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')
    
    ax[1].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR), label = round(TR,2))
    ax[1].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')

    ax[2].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[1], label = round(TR,2))
    ax[2].set_ylabel(r'$\frac{\dot{W}_{avail}}{\dot{m}c_pT_{t2}}$')
    ax[3].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[2], label = 'CompWork', color = 'k')
    #ax[3].plot(PR, TR-Work_per_flow(eta_c,eta_t,TR,PR, turb_comp_works=True)[3], label = r'$(T_{t4} - T_{t3})/T_{t2}$', color = 'r')
    ax[3].set_ylabel(r'$\frac{\dot{W}_{comp}}{\dot{m}c_pT_{t2}}$')
    
for a in ax:
    a.grid(alpha = 0.8)
    a.set_xlabel(r'Pressure ratio $\pi = \frac{P_{t3}}{P_{t2}}}$')
    lines = a.get_lines()
    l1=lines[-1]
    labelLine(l1,55,label=r'$TR=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 10)
    labelLines(lines[:-1], xvals=(20, 60), align=True,fontsize = 10)
    
plt.tight_layout()

In [ ]:

TurbEff_slider = widgets.FloatSlider(
    value=0.95,
    min=0.3,
    max=1.0,
    step=0.05,
    description=r"$\eta_{comp/turb}$:",
    readout_format='.2f',
    fontsize = 30,
)
def f(x):
    # Vary comp Eff
    PR = np.linspace(1,60,100)

    fig, ax = plt.subplots(2,1,figsize=(8,6),dpi =150, sharex = False)

    cc = cycler(linestyle=['-', '--', '-.'])

    eta_c = x
    eta_t = x
    eta_pt = 0.95
    # for a in ax:
    #     a.set_prop_cycle(cc)
    TR = 5
#     for eta_c in np.linspace(0.7,0.95,5):

    ax[0].plot(PR, ThermalEff(eta_c,eta_t,TR,PR), label = round(eta_c,2))
    ax[0].set_ylabel(r'$\eta_{thermal}$')
    ax[0].set_ylim([0,0.8])

    ax[1].plot(PR, Work_per_flow(eta_c,eta_t,TR,PR), label = round(eta_c,2) )
    ax[1].set_ylabel(r'$\frac{\dot{W}_{net}}{\dot{m}c_pT_{t2}}$')
    ax[1].set_xlabel(r'Pressure ratio $\pi = \frac{P_{t3}}{P_{t2}}}$')
    ax[1].set_ylim([0,2])
    for a in ax:
        a.grid(alpha = 0.8)
        
#         lines = a.get_lines()
#         l1=lines[-1]
#         labelLine(l1,55,label=r'$\eta_c=${}'.format(l1.get_label()),ha='center',va='center',align = True, fontsize = 9,
#                   bbox=dict(fc = "white", ec = "none",pad = 0))
#         labelLines(lines[:-1], xvals=(10, 60), align=True,fontsize = 9,
#                    bbox=dict(fc = "white", ec = "none", pad = 0))

    title = r"Comp/Turb. $\eta$ = "+str(x)
    fig.suptitle(title)

In [ ]:
interact(f, x = TurbEff_slider)